<a href="https://colab.research.google.com/github/Harsha07E9/password-manager/blob/main/password_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import getpass
import base64
import secrets
import string
from cryptography.fernet import Fernet
from collections import Counter

# Generate encryption key and save it to a file if it doesn't exist
def load_key():
    if not os.path.exists("secret.key"):
        key = Fernet.generate_key()
        with open("secret.key", "wb") as key_file:
            key_file.write(key)
    else:
        with open("secret.key", "rb") as key_file:
            key = key_file.read()
    return key

# Encrypt data
def encrypt_data(data, key):
    f = Fernet(key)
    encrypted = f.encrypt(data.encode())
    return encrypted

# Decrypt data
def decrypt_data(encrypted_data, key):i
    f = Fernet(key)
    decrypted = f.decrypt(encrypted_data).decode()
    return decrypted

# Generate a random, strong password with a minimum length check
def generate_password(length=12):
    if length < 8:  # Minimum length requirement for strong passwords
        print("Password length should be at least 8 characters.")
        return None

    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(secrets.choice(characters) for _ in range(length))
    return password

# Assess password strength with basic checks
def assess_password_strength(password):
    if len(password) < 8:
        return "Weak: Password is too short."
    if not any(char.isdigit() for char in password):
        return "Weak: Password should include at least one digit."
    if not any(char.isupper() for char in password):
        return "Weak: Password should include at least one uppercase letter."
    if not any(char.islower() for char in password):
        return "Weak: Password should include at least one lowercase letter."
    if not any(char in string.punctuation for char in password):
        return "Weak: Password should include at least one special character."

    # AI-Enhanced checks
    sequential_patterns = ["123", "abc", "qwerty", "password", "111"]
    if any(pattern in password.lower() for pattern in sequential_patterns):
        return "Weak: Contains an easily guessable sequence."

    char_counts = Counter(password)
    if max(char_counts.values()) > len(password) // 2:
        return "Weak: Contains too many repetitive characters."

    return "Strong: Password meets complexity requirements."

# Add a new password to the manager
def add_password(account, password, key):
    encrypted_password = encrypt_data(password, key)
    if os.path.exists("passwords.json"):
        with open("passwords.json", "r") as file:
            passwords = json.load(file)
    else:
        passwords = {}

    passwords[account] = base64.urlsafe_b64encode(encrypted_password).decode()

    with open("passwords.json", "w") as file:
        json.dump(passwords, file)

# Retrieve a password from the manager
def get_password(account, key):
    if not os.path.exists("passwords.json"):
        print("No passwords saved.")
        return

    with open("passwords.json", "r") as file:
        passwords = json.load(file)

    if account in passwords:
        encrypted_password = base64.urlsafe_b64decode(passwords[account])
        password = decrypt_data(encrypted_password, key)
        print(f"Password for {account}: {password}")
    else:
        print("Account not found.")

# Main function
def main():
    key = load_key()

    # Master password prompt
    master_password = getpass.getpass("Enter master password: ")
    stored_master_password = "manoj"  # Set a fixed master password for now

    if master_password != stored_master_password:
        print("Incorrect master password.")
        return

    while True:
        print("\nOptions:")
        print("1. Add a new password")
        print("2. Retrieve a password")
        print("3. Generate a strong password")
        print("4. Exit")

        choice = input("Choose an option: ")

        if choice == "1":
            account = input("Enter account name: ")

            while True:
                password = input("Enter password (leave blank to auto-generate): ")

                # Auto-generate if left blank
                if not password:
                    password = generate_password()
                    print(f"Generated password: {password}")

                # Check password strength
                assessment = assess_password_strength(password)
                print(f"Password Strength: {assessment}")

                if "Weak" not in assessment:  # Check if the password is strong
                    add_password(account, password, key)
                    print("Password saved successfully.")
                    break  # Exit the loop if password is strong
                else:
                    print("Consider using a stronger password.")

        elif choice == "2":
            account = input("Enter account name to retrieve password: ")
            get_password(account, key)

        elif choice == "3":
            length = int(input("Enter desired password length: "))
            generated_password = generate_password(length)
            if generated_password:  # Check if a valid password was generated
                print(f"Generated password: {generated_password}")
                # Assess the generated password strength
                assessment = assess_password_strength(generated_password)
                print(f"Generated Password Strength: {assessment}")

        elif choice == "4":
            print("Exiting.")
            break

        else:
            print("Invalid option. Please try again.")

if __name__ == "__main__":  # Fixed typo here
    main()